In [1]:
import json
import numpy as np
import matplotlib
import re

In [ ]:
def extract_run_time(trial):
    for line in trial.split('\n'):
        if 'Test took' in line:
            time_str = re.findall("\d+\.\d+", line)[0]
            time = float(time_str)
            # compute units
            unit = line[10 + len(time_str):]
            if unit == 'ms':
                time *= 1000
            if unit == 'us':
                time *= 1_000_000
            return time
    return 0.0


def extract_experiments(file_name):
    result = {}
    with open(file_name) as f:
        parsed_json = json.load(f)
        for expt in parsed_json.keys():
            trials = parsed_json[expt]
            trial_times = []
            for trial in trials:
                trial_times.append(extract_run_time(trial))
            avg = np.average(np.array(trial_times))
            result[expt] = avg

    return result
